In [10]:
import items, enemies, actions, world, npc
from player_file import player

In [11]:
class map_tile:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def intro_text(self):
        raise NotImplementedError()

    def modify_player(self, player):
        raise NotImplementedError()
    
    def adjacent_moves(self):
        '''Returns all move actions for adjacent tiles'''
        moves = []
        if world.tile_exists(self.x + 1, self.y):
            moves.append(actions.MoveEast())
        if world.tile_exists(self.x - 1, self.y):
            moves.append(actions.MoveWest())
        if world.tile_exists(self.x, self.y - 1):
            moves.append(actions.MoveNorth())
        if world.tile_exists(self.x, self.y + 1):
            moves.append(actions.MoveSouth())
        return moves
    
    def available_actions(self):
        '''Returns all of the available actions in this room.'''
        moves = self.adjacent_moves()
        moves.append(actions.view_inventory())
        
        if player.hp < 100:
            moves.append(actions.Heal())
        
        return moves


In [18]:
class start_room(map_tile):
    def intro_text(self):
        return '''
        
        I am in a room. It looks like my bedroom.
        It's not my bedroom.
        
        '''
    
    def modify_player(self, player):
        #Room has no action on the player
        pass


In [15]:
class enemy_room(map_tile):
    def __init__(self, x, y, enemy):
        self.enemy = enemy
        super().__init__(x, y)
        
    def modify_player(self, player):
        if self.enemy.is_alive():
            player.hp = player.hp - self.enemy.damage
            print("The enemy does {} damage. I have {} HP remaining".format(self.enemy.damage, player.hp))
            
        if not self.enemy.is_alive() and self.enemy.type_boss == True:
            player.inventory.append(self.enemy.weapon)
            print('The enemy is dead. I take the weapon they weilded against me as my own. I now use {}.'.format(self.enemy.weapon))
            
    def available_actions(self):
        if self.enemy.is_alive():
            return [actions.Flee(tile=self), actions.Attack(enemy=self.enemy)]
        else:
            return self.adjacent_moves()


In [16]:
class loot_room(map_tile):
    def __init__(self, x, y, item):
        self.item = item
        super().__init__(x, y)
        
    def add_loot(self, player):
        player.inventory.append(self.item)
        
    def modify_player(self, player):
        self.add_loot(player)


In [20]:
class trader_room(map_tile):
    def __init__(self, x, y):
        self.trader = npc.trader()
        super().__init__(x, y)
        
    def trade(self, buyer, trader):
        for i, item in enumerate(trader.inventory, 1):
            print("{}. {} - {} Gold".format(i, item.name, item.value))
        
        while True:
            user_input = input("Choose an item or press \"q\" to exit: ")
            
            if user_input == 'q':
                return
            
            else:
                try:
                    choice = int(user_input)
                    to_swap = seller.inventory[choice - 1]
                    self.swap(seller, buyer, to_swap)
                
                except ValueError:
                    print("Invalid choice!")
                    
    def swap(self, seller, buyer, item):
        
        if item.value > buyer.gold:
            print("That's too expensive")
            return
        
        seller.inventory.remove(item)
        buyer.inventory.append(item)
        
        seller.gold = seller.gold + item.value
        buyer.gold = buyer.gold - item.value
        print("Trade complete!")
        
    def check_if_trade(self, player):
        while True:
            print("Would you like to (b)uy, (s)ell, or (q)uit?")
            user_input = input()
            
            if user_input in ['Q', 'q']:
                return
            
            elif user_input in ['B', 'b']:
                print("Here's whats available to buy: ")
                self.trade(buyer=player, seller=self.trader)
            
            elif user_input in ['S', 's']:
                print("Here's whats available to sell: ")
                self.trade(buyer=self.trader, seller=player)
            
            else:
                print("Invalid choice!")
                
    def intro_text(self):
        return '''
        
        I see a frail man sitting in a cardboard box. 
        He has gold coins scattered all over the floor.
        He beckons to me.
        Looks like he wants to trade.
        
        '''
